In [1]:
import numpy as np
import SimpleITK as sitk
import SimpleITK._SimpleITK as _sitk
import os
import pandas as pd

In [2]:
input_dir = '/mnt/netcache/diag/silvan/Projects/CardiacOCT/prepared_data5/'
output_dir = '/mnt/netcache/diag/silvan/Projects/CardiacOCT/nnunet_data5/'

In [3]:
def create_circular_mask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center >= radius
    mask = np.expand_dims(mask,0)
    return mask

In [4]:
df = pd.read_csv(input_dir + 'Randomizatie_first_analysis.csv')

IDs = os.listdir(input_dir + 'images')

training_patients = []
testing_patients = []

for row in df.itertuples():
    patient_ID = row.Site
    set_ID = row.Arm
    if set_ID == 'Training':
        training_patients.append(patient_ID)
    elif set_ID == 'Validation':
        testing_patients.append(patient_ID)

train_IDs = []
testing_IDs = []

for frame_ID in IDs:
    if np.isin(frame_ID.split('_')[0], training_patients):
        train_IDs.append(frame_ID)

    elif np.isin(frame_ID.split('_')[0], testing_patients):
        testing_IDs.append(frame_ID)
        

In [9]:
loc = input_dir + 'images/'
loc_ann = input_dir + 'annotations/'


mask =  create_circular_mask(512, 512, center=[256,256], radius=250)

def prepare_data(list_IDs, loc, loc_ann, loc_out, loc_ann_out, test = False):
    for index in range(len(list_IDs)):
        ID = list_IDs[index]
        print(ID)
        # Load data and get label

        X_image = sitk.ReadImage(loc + ID)
        X = sitk.GetArrayFromImage(X_image).astype('uint8')
        y_image = sitk.ReadImage(loc_ann + ID)
        y_hat = sitk.GetArrayFromImage(y_image).astype('uint8')

        print(X.shape)
        print(y_hat.shape)
        if y_hat.shape[1]==572:
            y = np.zeros((1,576,576)).astype('uint8')
            y[0,:572,:572] = y_hat
        else:
            y = y_hat

        X[:,mask]=0
        y[mask]=0


        X_spacing = X_image.GetSpacing()
        X_spacing_list = list(X_spacing)
        X_spacing_list[2] = 2.0
        X_spacing = tuple(X_spacing_list)
        X_direction = X_image.GetDirection()
        X_origin = X_image.GetOrigin()

        y_spacing = y_image.GetSpacing()
        
        y_spacing_list = list(y_spacing)
        y_spacing_list[2] = 2.0
        y_spacing = tuple(y_spacing_list)
        y_direction = y_image.GetDirection()
        y_origin = y_image.GetOrigin()
#         print(y_spacing)
#         print(y_origin)
#         print(y_direction)
        #print(y_spacing)
        
        # Flip data
        if test:
            n_rot = 1
        else:
            n_rot = 4       

        for nn in range(n_rot):

            if nn==0:
                X_out = X.copy()
                y_out = y.copy()
            if nn == 1:
                X_out = np.flip(np.transpose(X,(0,1,3,2)),3).copy()
                y_out = np.flip(np.transpose(y,(0,2,1)),2).copy()
            elif nn == 2:
                X_out = np.flip(np.flip(X,2),3).copy()
                y_out = np.flip(np.flip(y,1),2).copy()
            elif nn == 3:
                X_out = np.transpose(np.flip(X,3),(0,1,3,2)).copy()
                y_out = np.transpose(np.flip(y,2),(0,2,1)).copy()

            X_out = np.transpose(np.squeeze(X_out), (1,2,0))
            y_out = np.transpose(y_out, (1,2,0))

#                 print(y_out.shape)
            output = sitk.Image(X_out.shape[::-1], sitk.sitkUInt8, 1)
            output.SetSpacing(y_spacing)
            output.SetOrigin(y_origin)
            output.SetDirection(y_direction)
            _sitk._SetImageFromArray(X_out.tostring(), output)

            sitk.WriteImage(output, loc_out + ID[:-4] + '_' + str(nn) + '.mha', True)

            output = sitk.Image(y_out.shape[::-1], sitk.sitkUInt8, 1)
            output.SetSpacing(y_spacing)
            output.SetOrigin(y_origin)
            output.SetDirection(y_direction)
            _sitk._SetImageFromArray(y_out.tostring(), output)
            sitk.WriteImage(output, loc_ann_out + ID[:-4] + '_' + str(nn) + '.mha', True)


In [10]:
loc = input_dir + 'images/'
loc_ann = input_dir + 'annotations/'
loc_out = output_dir + 'imagesTr/'
loc_ann_out = output_dir + 'labelsTr/'

prepare_data(train_IDs, loc, loc_ann, loc_out, loc_ann_out, test = False)

# loc = input_dir + 'images/'
# loc_ann = input_dir + 'annotations/'
# loc_out = output_dir + 'imagesTs/'
# loc_ann_out = output_dir + 'labelsTs/'

# prepare_data(testing_IDs, loc, loc_ann, loc_out, loc_ann_out, test = True)

NLD-ISALA-0070_frame_321.mha
(3, 1, 512, 512)
(1, 512, 512)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:78: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


NLD-ISALA-0006_frame_41.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0076_frame_41.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0090_frame_401.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-HMC-0002-RCA_frame_41.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0059_frame_81.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0029_frame_481.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0097_frame_71.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0090_frame_406.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0084-MO2_frame_441.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0029_frame_321.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0073_frame_81.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0094-LAD_frame_498.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0084_frame_401.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0079_frame_1.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0012_frame_344.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0076_frame_1.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0095_frame_133.mha
(3, 1, 512, 512)
(1,

NLD-AMPH-0017-LAD_frame_463.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-HMC-0009-RCAp_frame_450.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0061_frame_281.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0007_frame_181.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0057_frame_321.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-HMC-0009-RCAd_frame_161.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-HMC-0009-RCAp_frame_321.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0072_frame_41.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0095-RCx_frame_401.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-HMC-0002-RPL_frame_1.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0096_frame_401.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0017-LAD_frame_121.mha
(3, 1, 512, 512)
(1, 512, 512)
EST-NEMC-0027-RCA_frame_152.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0089_frame_161.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0095-LAD_frame_241.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0075_frame_281.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0088-RCx_fr

NLD-ISALA-0008_frame_361.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0076_frame_521.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-HMC-0002-RPL_frame_241.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0013_frame_135.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0058_frame_161.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0029_frame_41.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0062_frame_1.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0092_frame_481.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0062_frame_281.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0065_frame_121.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-HMC-0009-RCAd_frame_1.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0096_frame_201.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0065-MO1_frame_361.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0076_frame_81.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-HMC-0008_frame_1.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0093_frame_241.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0045-RCA_frame_201.mha
(3, 1, 512, 

NLD-AMPH-0072_frame_201.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0084_frame_161.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0062_frame_321.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0091_frame_481.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0084-MO2_frame_121.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0095_frame_201.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0090_frame_241.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0007_frame_201.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0085_frame_59.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0017-LAD_frame_41.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0076_frame_481.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0012_frame_361.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0085_frame_321.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0066_frame_201.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0094-LAD_frame_361.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0063_frame_241.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0090_frame_161.mha
(3, 1, 512, 512

NLD-RADB-0095_frame_321.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0066_frame_481.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0066_frame_24.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0011_frame_193.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0097_frame_81.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0092_frame_41.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0076_frame_201.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0051-LAD_frame_81.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0085_frame_121.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0073_frame_241.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0005_frame_1.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0094-LAD_frame_241.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0063_frame_361.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0062_frame_81.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0066_frame_321.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0095_frame_481.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0086_frame_135.mha
(3, 1, 512, 512)
(1, 5

NLD-AMPH-0062_frame_41.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0006_frame_121.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0058_frame_521.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-HMC-0008_frame_41.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0097_frame_401.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0045-RCA_frame_81.mha
(3, 1, 512, 512)
(1, 512, 512)
EST-NEMC-0027-RCA_frame_206.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0088_frame_161.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0005_frame_401.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0097_frame_51.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0076_frame_161.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-HMC-0009-RCAp_frame_468.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0029_frame_259.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0084_frame_201.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0070_frame_281.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0063_frame_41.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0029_frame_405.mha
(3, 1, 512, 512

NLD-AMPH-0066_frame_41.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0059_frame_201.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0045-RCA_frame_416.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0065-LAD_frame_106.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0013_frame_170.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0086_frame_150.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0070_frame_241.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0061_frame_353.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0063_frame_81.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0085_frame_401.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0072_frame_61.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0084-MO2_frame_281.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0062_frame_81.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0096_frame_201.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-RADB-0084-MO1_frame_121.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-AMPH-0075_frame_481.mha
(3, 1, 512, 512)
(1, 512, 512)
NLD-ISALA-0095-LAD_frame_441.mha
(3, 1, 

In [89]:
print(sitk.GetArrayFromImage(sitk.ReadImage(loc_out + "NLD-ISALA-0070_frame_321_0.mha")).shape)

(3, 1, 512, 512)


In [24]:
img = io.imread(loc_out + "NLD-ISALA-0070_frame_321_0.mha")
img.shape
file_name = loc_out + "NLD-ISALA-0070_frame_321_0.mha"
convert_2d_image_to_nifti(file_name, file_name[:-4], is_seg=False)

In [3]:
import numpy as np
from batchgenerators.utilities.file_and_folder_operations import *
from nnunet.dataset_conversion.utils import generate_dataset_json
from nnunet.paths import nnUNet_raw_data, preprocessing_output_dir
from nnunet.utilities.file_conversions import convert_2d_image_to_nifti



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

nnUNet_raw_data_base is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
RESULTS_FOLDER is not defined and nnU-Net cannot be used for training or inference. If this is not in

In [4]:
!nnUNet_predict -i /mnt/netcache/diag/silvan/Projects/CardiacOCT_nnunet/nnUNet_raw_data/Task101_CardiacOCT/imagesTs/ -o /mnt/netcache/diag/silvan/Projects/CardiacOCT_nnunet/processor_test/output -t 101 -m 2d



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

nnUNet_raw_data_base is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
RESULTS_FOLDER is not defined and nnU-Net cannot be used for training or inference. If this is not in